In [ ]:
import copy
import itertools
import pandas as pd
import numpy as np
import statistics
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
# set path to datasets
PATH_TO_RPKM_DATA = "./data/CCLE_RNAseq_genes_rpkm_20180929.gct"
PATH_TO_ANTICANCER_DRUG_DATA = "./data/CCLE_NP24.2009_Drug_data_2015.02.24.csv"

In [ ]:
# reading RPKM data
# first two rows and first two columns are irrelevant
# index_col can be used to set the row names
RPKMData = pd.read_csv(PATH_TO_RPKM_DATA, sep = '\t', skiprows = 2, index_col = 0)

drugData = pd.read_csv(PATH_TO_ANTICANCER_DRUG_DATA, delimiter=',')

In [ ]:
# identifying compounds in dataset
allCompounds = drugData['Compound'].unique()
print(allCompounds)

# separate data by compound 
drugDataByCompound = {compound : drugData.loc[drugData['Compound'] == compound] for compound in allCompounds}
print(drugDataByCompound[allCompounds[0]])

In [ ]:
# sort data per compound by IC50
for compound in allCompounds:
    sortData = copy.deepcopy(drugDataByCompound[compound])
    sortData.sort_values(by = ['IC50 (uM)'], inplace = True)
    drugDataByCompound[compound] = sortData
print(drugDataByCompound[allCompounds[0]])

In [ ]:
fig, axes = plt.subplots(4, 6, figsize = (12, 12))
fig.set_facecolor('w')
for i in range(4):
    for j in range(6):
        compound = allCompounds[i*6 + j]
        axes[i, j].plot(drugDataByCompound[compound]['CCLE Cell Line Name'], drugDataByCompound[compound]['IC50 (uM)'], color = 'b')
        axes[i, j].axhline(y = drugDataByCompound[compound]['IC50 (uM)'].mean(), color = 'r')
        axes[i, j].text(0, drugDataByCompound[compound]['IC50 (uM)'].mean() + 0.25, str(drugDataByCompound[compound]['IC50 (uM)'].mean()), color = 'r', fontsize = 8.5)
        axes[i, j].set_title(compound)
plt.setp(plt.gcf().get_axes(), xticks=[])

In [ ]:
# extract IC50 data for a given compound
compound = 'RAF265'
ic50 = pd.Series(list(drugDataByCompound[compound]['IC50 (uM)']), index = list(drugDataByCompound[compound]['CCLE Cell Line Name']))
print(ic50)

In [ ]:
# limit data to samples with IC50 data and RPKM data
all_samples = list(set(RPKMData.columns) & set(ic50.index))
ic50 = ic50[all_samples]
print(ic50)

In [ ]:
# convert IC50 data into resistant/sensitive labels based on mean()
drug_response = pd.Series(['Resistant' if item > ic50.mean() else 'Sensitive' for i, item in ic50.iteritems()], index = ic50.index)
print(drug_response)

In [ ]:
# apply a variance based feature reduction (removing bottom 40% lowest variance)
filtered_RPKMData = RPKMData[all_samples]
filtered_RPKMData = filtered_RPKMData[filtered_RPKMData.var(axis = 1) > RPKMData[all_samples].var(axis = 1).quantile(q = 0.4)]

## evaluate adding tumor type as a feature
unique_tumor_types = set(['_'.join(sample.split('_')[1:]) for sample in all_samples])
encoding_map = {tumor_type : index for index, tumor_type in enumerate(unique_tumor_types)}
tumor_type_df = pd.DataFrame({'TumorType' : {sample : encoding_map['_'.join(sample.split('_')[1:])] for sample in all_samples}}).transpose()
filtered_withtype_RPKMData = pd.concat([filtered_RPKMData, tumor_type_df])

In [ ]:
# randomly assign samples to bins
sample_bins = {}
n_bins = 30
random_seed = 1234
drug_response = drug_response.sample(frac = 1, random_state = random_seed)
for i, sample in enumerate(drug_response.index):
    sample_bins.setdefault(i%n_bins,[]).append(sample)

In [ ]:
# unfiltered data
total_correct = 0
total_tested = 0
for test_bin in range(n_bins):
    train_samples = list(itertools.chain.from_iterable([sample_bins[i] if i != test_bin else [] for i in sample_bins.keys()]))
    test_samples = sample_bins[test_bin]
    train_labels = drug_response[train_samples]
    train_RPKM = RPKMData[train_samples].transpose()
    test_labels = drug_response[test_samples]
    test_RPKM = RPKMData[test_samples].transpose()

    train_rf = RandomForestClassifier(n_estimators = 1000, random_state = random_seed)
    train_rf.fit(train_RPKM, train_labels)
    test_predictions = pd.Series(train_rf.predict(test_RPKM), index = test_RPKM.index)

    total_correct += sum(test_predictions == test_labels)
    total_tested += len(test_predictions)

print(total_correct/total_tested)

In [ ]:
# with filtered data
total_correct = 0
total_tested = 0
for test_bin in range(n_bins):
    train_samples = list(itertools.chain.from_iterable([sample_bins[i] if i != test_bin else [] for i in sample_bins.keys()]))
    test_samples = sample_bins[test_bin]
    train_labels = drug_response[train_samples]
    train_RPKM = filtered_RPKMData[train_samples].transpose()
    test_labels = drug_response[test_samples]
    test_RPKM = filtered_RPKMData[test_samples].transpose()

    train_rf = RandomForestClassifier(n_estimators = 1000, random_state = random_seed)
    train_rf.fit(train_RPKM, train_labels)
    test_predictions = pd.Series(train_rf.predict(test_RPKM), index = test_RPKM.index)

    total_correct += sum(test_predictions == test_labels)
    total_tested += len(test_predictions)

print(total_correct/total_tested)

In [ ]:
# filtered data with tumor type as feature
total_correct = 0
total_tested = 0
for test_bin in range(n_bins):
    train_samples = list(itertools.chain.from_iterable([sample_bins[i] if i != test_bin else [] for i in sample_bins.keys()]))
    test_samples = sample_bins[test_bin]
    train_labels = drug_response[train_samples]
    train_RPKM = filtered_withtype_RPKMData[train_samples].transpose()
    test_labels = drug_response[test_samples]
    test_RPKM = filtered_withtype_RPKMData[test_samples].transpose()

    train_rf = RandomForestClassifier(n_estimators = 1000, random_state = random_seed)
    train_rf.fit(train_RPKM, train_labels)
    test_predictions = pd.Series(train_rf.predict(test_RPKM), index = test_RPKM.index)

    total_correct += sum(test_predictions == test_labels)
    total_tested += len(test_predictions)

print(total_correct/total_tested)

In [ ]:
for train_bins_count in [1, 2, 5, 10, 20, 29]:
    total_correct = 0
    total_tested = 0
    bin_interval = min(train_bins_count, n_bins - train_bins_count)
    for current_interval in range(int(n_bins/bin_interval)):
        if train_bins_count == bin_interval:
            train_samples = list(itertools.chain.from_iterable([sample_bins[i] if i in range(current_interval*bin_interval, (current_interval + 1)*bin_interval) else [] for i in sample_bins.keys()]))
            test_samples = list(itertools.chain.from_iterable([sample_bins[i] if i not in range(current_interval*bin_interval, (current_interval + 1)*bin_interval) else [] for i in sample_bins.keys()]))
        else:
            train_samples = list(itertools.chain.from_iterable([sample_bins[i] if i not in range(current_interval*bin_interval, (current_interval + 1)*bin_interval) else [] for i in sample_bins.keys()]))
            test_samples = list(itertools.chain.from_iterable([sample_bins[i] if i in range(current_interval*bin_interval, (current_interval + 1)*bin_interval) else [] for i in sample_bins.keys()]))
        # add a step to remove any imputed samples from the test array
        train_labels = drug_response[train_samples]
        train_RPKM = filtered_RPKMData[train_samples].transpose()
        test_labels = drug_response[test_samples]
        test_RPKM = filtered_RPKMData[test_samples].transpose()

        train_rf = RandomForestClassifier(n_estimators = 1000, random_state = random_seed)
        train_rf.fit(train_RPKM, train_labels)
        test_predictions = pd.Series(train_rf.predict(test_RPKM), index = test_RPKM.index)

        total_correct += sum(test_predictions == test_labels)
        total_tested += len(test_predictions)
    print((train_bins_count, total_correct/total_tested))
    
# next steps here, impute labels for samples in RNAseq dataset without labels then see the effect of using those data points for training
# (include this data in the randomization of binning, and use this data to train the model, but don't use this data in the calculation of accuracy)

In [ ]:
# set up training and testing datasets

test_bin = 0 # iterate
train_samples = list(itertools.chain.from_iterable([sample_bins[i] if i != test_bin else [] for i in sample_bins.keys()]))
train_labels = drug_response[train_samples]
train_RPKM = RPKMData[train_samples].transpose()

test_samples = sample_bins[test_bin]
test_labels = drug_response[test_samples]
test_RPKM = RPKMData[test_samples].transpose()

# copy train samples
# for loop and crop out left of first instance of '_'
# append to the training dataframe
cancer_train_samples = train_samples
for i in range(len(cancer_train_samples)):
    cancer_type = cancer_train_samples[i][cancer_train_samples[i].find('_') + 1:]
    cancer_train_samples[i] = cancer_types_map.index(cancer_type)
train_RPKM["Cancer"] = cancer_train_samples

cancer_test_samples = test_samples
for i in range(len(cancer_test_samples)):
    cancer_type = cancer_test_samples[i][cancer_test_samples[i].find('_') + 1:]
    cancer_test_samples[i] = cancer_types_map.index(cancer_type)
test_RPKM["Cancer"] = cancer_test_samples

In [ ]:
# model
train_rf = RandomForestClassifier(n_estimators = 256, random_state = random_seed)
train_rf.fit(train_RPKM, train_labels)
test_predictions = pd.Series(train_rf.predict(test_RPKM), index = test_RPKM.index)

# compares predictions against actual labels to get accuracy
print(sum(test_predictions == test_labels)/len(test_predictions))

In [ ]:
#only finds the important features of the training data, not everything
feature_importances = pd.Series(train_rf.feature_importances_, index = train_RPKM.columns) # keep track of each feature importances for each iteration of test bins and average
feature_importances = feature_importances[feature_importances > 0]
important_features = list(feature_importances.index.values)

#find features that are greater than 0 *implies importance to the model*
feature_drops = list(np.setdiff1d(list(train_RPKM.columns), important_features))
feature_drops.remove("Cancer") # make sure to keep the cancer feature

In [ ]:
# clear any of the features we deemed not important
important_train_RPKM = train_RPKM.drop(columns=feature_drops)

important_test_RPKM = test_RPKM.drop(columns=feature_drops)

In [ ]:
# model IMPORTANT FEATURES ONLY
important_train_rf = RandomForestClassifier(n_estimators = 256, random_state = random_seed)
important_train_rf.fit(important_train_RPKM, train_labels)
important_test_predictions = pd.Series(important_train_rf.predict(important_test_RPKM), index = important_test_RPKM.index)

# compares predictions against actual labels to get accuracy
print(sum(important_test_predictions == test_labels)/len(important_test_predictions))

In [ ]:
feature_drops = list(np.setdiff1d(list(RPKMData.index.values), important_features))
important_RPKMData = RPKMData.drop(feature_drops).drop(columns=['Description'])
important_RPKMData

In [ ]:
len(important_RPKMData.iloc[0])

In [ ]:
normalized_RPKMData = important_RPKMData
for r in range(len(normalized_RPKMData)):
    row_sum = sum(list(normalized_RPKMData.iloc[r]))
    mean = row_sum / len(normalized_RPKMData.iloc[r])
    variance = statistics.variance(list(normalized_RPKMData.iloc[r]))
    for c in range(len(normalized_RPKMData.iloc[r])):
        normalized_RPKMData.iloc[r, c] = (normalized_RPKMData.iloc[r, c] - mean) / variance
        
normalized_RPKMData

In [ ]:
plt.figure(figsize = (20,20)) # normalize the data *subtract mean and divide by variance*
ax = sns.heatmap(normalized_RPKMData.to_numpy(), linewidths = 0.01)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(RPKMData.)

In [ ]:
np.max(normalized_RPKMData.to_numpy())


In [ ]:
test_bin = 0
train_samples = list(itertools.chain.from_iterable([sample_bins[i] if i != test_bin else [] for i in sample_bins.keys()]))
test_samples = sample_bins[test_bin]
train_labels = drug_response[train_samples]
train_RPKM = RPKMData[train_samples].transpose()
test_labels = drug_response[test_samples]
test_RPKM = RPKMData[test_samples].transpose()

train_svm = svm.SVC()
train_svm.fit(train_RPKM, train_labels)
test_predictions = train_svm.predict(test_RPKM)